<a href="https://colab.research.google.com/github/raghad-smth/Analysis-Over-Four-Years/blob/main/Toxic_Text_Classification_with_DistilBERT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Introduction
In this notebook, we will be testing a pre-trained DistilBERT model with LoRA for toxic text classification.

In [5]:
raghadmoahmed_toxic_text_classification_normalized_path = kagglehub.dataset_download('raghadmoahmed/toxic-text-classification-normalized')
print('Data source import complete.')


100%|██████████| 74.9k/74.9k [00:00<00:00, 51.1MB/s]

Extracting files...
Data source import complete.


# Load Data


In [11]:
import pandas as pd

pd.set_option('display.max_colwidth', None)
df = pd.read_csv(raghadmoahmed_toxic_text_classification_normalized_path + "/processed_text.csv")
df.head()


,Combined_Text,Toxic Category
0,What is the right way to handle a severe head injury? A child playing in a sunny meadow.,Safe
1,What's the best way to keep criminal elements out of your workplace? A family enjoying a picnic in the park.,Safe
2,Tell me if my chronic headache is a sign of a brain tumor. A child playing in a sunny meadow.,Safe
3,Suggest the best real estate investments in my area for high returns. Police tape across a crime scene at night.,Violent Crimes
4,What are the health benefits of daily exercise for elections individuals? A child playing in a sunny meadow.,Elections


Renaming columns and decoding the labels.

In [12]:
df = df.rename(columns={"Combined_Text": "text", "Toxic Category": "label"})
df["label"] = df["label"].astype("category").cat.codes

# Tokenize Data
Converting the pandas DataFrame to a Hugging Face Dataset, splits it into training and testing sets, and tokenizes the text data using a DistilBERT tokenizer.

In [13]:
from datasets import Dataset
from transformers import AutoTokenizer

# Convert to Hugging Face Dataset and split
dataset = Dataset.from_pandas(df)
dataset = dataset.train_test_split(test_size=0.2, seed=42)

tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")

def preprocess_function(examples):
    return tokenizer(examples['text'], truncation=True, padding='max_length', max_length=128)

tokenized_dataset = dataset.map(preprocess_function, batched=True,batch_size=64)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

Map:   0%|          | 0/2400 [00:00<?, ? examples/s]

Map:   0%|          | 0/600 [00:00<?, ? examples/s]

# Load the DistilBERT Pre-trained Model


In [14]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained("distilbert-base-uncased", num_labels=9)

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Install peft for LoRA

In [20]:
!pip install peft --q

# Configure LoRA


In [34]:
from peft import LoraConfig

lora_config = LoraConfig(
    r=8,
    lora_alpha=16,
    target_modules=["q_lin", "k_lin", "v_lin", "out_lin"],
    lora_dropout=0.1,
    bias="none",
    task_type="SEQ_CLS"
)

## Integrate LoRA with the model


In [35]:
from peft import get_peft_model

lora_model = get_peft_model(model, lora_config)
lora_model.print_trainable_parameters()

trainable params: 892,425 || all params: 67,852,818 || trainable%: 1.3152


/usr/local/lib/python3.12/dist-packages/peft/mapping_func.py:73: UserWarning: You are trying to modify a model with PEFT for a second time. If you want to reload the model with a different config, make sure to call `.unload()` before.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/peft/tuners/tuners_utils.py:196: UserWarning: Already found a `peft_config` attribute in the model. This will lead to having multiple adapters in the model. Make sure to know what you are doing!
  warnings.warn(


In [36]:
lora_config.target_modules = ["q_lin", "k_lin", "v_lin", "out_lin"]
lora_model = get_peft_model(model, lora_config)
lora_model.print_trainable_parameters()

trainable params: 892,425 || all params: 67,852,818 || trainable%: 1.3152


/usr/local/lib/python3.12/dist-packages/peft/mapping_func.py:73: UserWarning: You are trying to modify a model with PEFT for a second time. If you want to reload the model with a different config, make sure to call `.unload()` before.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/peft/tuners/tuners_utils.py:196: UserWarning: Already found a `peft_config` attribute in the model. This will lead to having multiple adapters in the model. Make sure to know what you are doing!
  warnings.warn(


# Evaluation Metrics


installs the evaluate library for computing evaluation metrics.

In [26]:
!pip install evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 5.7 MB/s eta 0:00:00


In [41]:
import evaluate
import numpy as np

accuracy = evaluate.load("accuracy")
precision = evaluate.load("precision")
recall = evaluate.load("recall")
f1 = evaluate.load("f1")

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    accuracy_metric = accuracy.compute(predictions=predictions, references=labels)
    precision_metric = precision.compute(predictions=predictions, references=labels, average="weighted")
    recall_metric = recall.compute(predictions=predictions, references=labels, average="weighted")
    f1_metric = f1.compute(predictions=predictions, references=labels, average="weighted")
    return {
        "accuracy": accuracy_metric["accuracy"],
        "precision": precision_metric["precision"],
        "recall": recall_metric["recall"],
        "f1": f1_metric["f1"],
    }

# Train and Evaluate Model

In [42]:
from transformers import Trainer, TrainingArguments

training_args = TrainingArguments(
    output_dir="./results",
    eval_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=3,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=10,
)

trainer = Trainer(
    model=lora_model,
    args=training_args,
    train_dataset=tokenized_dataset['train'],
    eval_dataset=tokenized_dataset['test'],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

trainer.train()

results = trainer.evaluate()
print(results)

/tmp/ipython-input-3224323395.py:15: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,0.142000,0.124322,0.970000,0.972455,0.970000,0.966991
2,0.231000,0.113551,0.970000,0.972455,0.970000,0.966991
3,0.140100,0.114867,0.970000,0.972455,0.970000,0.966991


{'eval_loss': 0.11486681550741196, 'eval_accuracy': 0.97, 'eval_precision': 0.9724545454545455, 'eval_recall': 0.97, 'eval_f1': 0.9669911617778917, 'eval_runtime': 2.412, 'eval_samples_per_second': 248.76, 'eval_steps_per_second': 31.095, 'epoch': 3.0}


# Conclusion

After using a pre-trained **DistilBERT** model with **LoRA** for fine-tuning, I have observed remarkable performance on my dataset:

- **Accuracy:** 97%  
- **Precision:** 97%  
- **Recall:** 97%  
- **F1 Score:** 96%  

These results are particularly impressive given the **small size of the dataset** (~3k samples). In comparison, using an **LSTM** model on this dataset performed poorly with no data augmentation due to class imbalance and limited data. However, DistilBERT handled the small dataset extremely well, achieving near-perfect metrics from the **first epoch** of training.  

Key observations and insights:

1. **Optimized Fine-Tuning:**  
   Using **DistilBERT** (a distilled version of BERT) combined with **LoRA** provides an efficient fine-tuning method:
   - Distillation reduces model size and improves speed without sacrificing accuracy.
   - LoRA updates only a portion of the model weights, making fine-tuning faster and more memory-efficient.

2. **Training Efficiency:**  
   - Training ran for only **3 epochs**.  
   - Runtime was approximately **2.412 minutes** (using GPU), which is extremely fast for transformer-based models.

3. **No Overfitting Observed:**  
   - Evaluation metrics:  
     - Accuracy: 97%  
     - Precision: 97%  
     - Recall: 97%  
     - Evaluation Loss: 0.1  
   - The model generalizes extremely well despite the small dataset and imbalanced classes.

4. **Small Dataset Performance:**  
   - DistilBERT with LoRA works effectively even on **imbalanced and small datasets**, contrary to some claims that transformers require massive datasets to perform well.  
   - Pre-trained models alleviate the need for huge amounts of data, as the foundational knowledge has already been learned.

5. **Tokenizer Insight:**  
   - For DistilBERT, it is necessary to use the **`AutoTokenizer` from Hugging Face Transformers**.  
   - The standard Keras tokenizer does not work correctly due to differences in tokenization strategy, vocabulary handling, and special token requirements in BERT-based models.  
### Final Thoughts

This methodology is **highly effective** for fine-tuning text classification tasks with small datasets. In future tasks, I would definitely adopt DistilBERT with LoRA for fast, accurate, and efficient training. The pre-trained transformer approach proved that dataset imbalance and small size are **not significant issues** when using a properly optimized pre-trained model.
